<a href="https://colab.research.google.com/github/shahnazumer/LCEL-DOCUMENT/blob/main/QDRANT_RAG_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### [Detailed](https://python.langchain.com/docs/use_cases/question_answering/)

# Retrieval Augmented Generation (RAG)

RAG is an AI framework for retrieving facts from an external knowledge base to ground large language models (LLMs) on the most accurate, up-to-date information and to give users insight into LLMs' generative process

In the context of this illustration, we will undertake the development of an AI chatbot, encompassing the utilization of LangChain, OpenAI, and Qdrant vector DB. The objective is to construct a chatbot endowed with the capacity to assimilate knowledge from the external environment through Retrieval Augmented Generation (RAG).

Upon completing this example, we anticipate having a fully operational chatbot and RAG pipeline capable of engaging in conversations and delivering informative responses based on an integrated knowledge base

# Prerequisites
Before we start building our chatbot, we need to install some Python libraries. Here's a brief overview of what each library does:

- **langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **Qdrant-client**: This is the official Qdrant client. We'll use it to interact with the Qdrant API / HOST and store our chatbot's knowledge base in a vector database.



You can install these libraries using pip like so:

In [ ]:
!pip install langchain==0.0.345 openai qdrant-client langchain-community langchain-core tiktoken


# Connecting to Qdrant

Initially, we will establish a connection between our data and the VectorStore, with Qdrant serving a pivotal role in this process. Qdrant functions as a vector similarity search engine, facilitating efficient retrieval and analysis of information based on vector representations. This integration enhances our ability to perform advanced similarity searches and extract meaningful insights from the connected data within the VectorStore framework


I'm using the code from https://github.com/raen-ai/QD_collections/blob/main/Playground.ipynb

In [ ]:
# We will be relying heavily on the LangChain library to bring together the different components needed for our model;

import qdrant_client
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import CharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client import models, QdrantClient
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
import os

os.environ["OPENAI_API_KEY"]

In [30]:
# create a qdrant instance

os.environ['QDRANT_HOST']
os.environ["QDRANT_API_KEY"]

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [ ]:
# read the file

doc ="/content/foundation.txt"
data=""

with open(doc,'r') as f:
    data = f.read()

In [31]:
# split the text into chunks
#create a function to return chunks
def get_chunks(text):
    text_splitter=CharacterTextSplitter(
        separator= "\n",
        chunk_size=500,
        chunk_overlap=100, # second chunk start  character from 800, overlap is used to stop loosing chunk
        length_function=len
    )

    chunks=text_splitter.split_text(text)
    return chunks

In [32]:
# get the chunks for the data
texts=get_chunks(data)

In [33]:
# creating a new collection

vectors_config=models.VectorParams(
    # depends on model, we can google dimension. 1536 for openai
    # we are using openai embedding, for that size is 1536
    size=1536,
    distance=models.Distance.COSINE)

client.create_collection(
    collection_name="Isaac Asimov Foundation",
    vectors_config=vectors_config,
)

True

In [36]:
# create a vector store object using langchain

# if we want to use any other embedding, we need to change size

os.environ["OPENAI_API_KEY"]

embeddings = OpenAIEmbeddings()

vector_store = Qdrant(
    client=client,
    collection_name="Isaac Asimov Foundation",
    embeddings=embeddings,
)

In [39]:
# add chunks to vector store
vector_store.add_texts(texts)

['4d22c83f88344aa591a4ca8a7fd99923',
 'f64bc0afbbbc418a97d8ba7a225a4b18',
 '5e26a9808e8b4f6898f64bd955285477',
 '2abf9dc397e144ca899266509dde043d',
 'ff277314ba10488cb37d6af827fd7fc1']

##Advanced Use Case: Retrieval Augmented Generation (RAG)

We've built a fully-fledged knowledge base. Now it's time to connect that knowledge base to our chatbot.

i have used the code from https://python.langchain.com/docs/expression_language/cookbook/retrieval

In [41]:
# create retriever & let's try passing a question through to a these vectorstore objects


retriever = vector_store.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [42]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
chain.invoke("Who is the creator of psychohistory in the Foundation series?")

'The creator of psychohistory in the Foundation series is Hari Seldon.'

In [48]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [49]:
chain.invoke({"question": "What role do technology and science play in the Foundation series?"})

'Technology and science play a significant role in the Foundation series. The series explores the concept of psychohistory, a mathematical discipline that combines history, sociology, and mathematics to predict the future of large populations. The protagonist, Hari Seldon, establishes the Foundation, a group of scientists and intellectuals, to preserve knowledge and guide the future. The Foundation relies on scientific means to shape the future and mitigate the collapse of the Galactic Empire. Additionally, the series emphasizes the power of science and reason, highlighting the interplay between individual actions and historical forces.'

# LCEL RAG with memory

I'll be using the code from https://python.langchain.com/docs/expression_language/cookbook/retrieval#with-memory-and-returning-source-documents

In [53]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [55]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [58]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [67]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [68]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [69]:
# First we add a step to load memory
# This adds a "memory" key to the input object

loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
        "standalone_question": {
            "question": lambda x: x["question"],
            "chat_history": lambda x: x["chat_history"],
        }
        | CONDENSE_QUESTION_PROMPT
        | model
        | StrOutputParser(),
    }

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [ ]:
inputs = {"question": "What is the Seldon Plan and why is it significant??"}
result = final_chain.invoke(inputs)
result